In [267]:
%run boilerplate_extract.ipynb

In [268]:
# Import required modules
import pandas as pd
import string
import datetime as dt

In [269]:
data_dict = read_data()

In [270]:
def clean_phone_number(phone: str):
    """
    To extract the phone number into numeric values
    """
    for char in phone:
        if char not in string.digits:
            phone = phone.replace(char, "")

    return phone

In [271]:
def check_postal_code(code: str):
    """
    To check if postal code is alphabet, return false; Else return true.
    """
    for char in code:
       if char in string.ascii_letters:
            return False
    return True

In [272]:
def transform_suppliers(data: pd.DataFrame):
    """
    To transform the suppliers table; return as a DataFrame.
    """
    # Add supplier_contact column
    data["supplier_contact"] = data["Phone"].apply(lambda x:0 if pd.isnull(x) else clean_phone_number(x))

    # Transform PostalCode column
    data["PostalCode"] = data["PostalCode"].apply(lambda x:x if check_postal_code(x) else 0)

    return data

In [273]:
def transform_shippers(data: pd.DataFrame):
    """
    To transform the shippers table; return as a DataFrame.
    """
    # Add shipper_contact column
    data["shipper_contact"] = data["Phone"].apply(lambda x:0 if pd.isnull(x) else clean_phone_number(x))

    return data

In [266]:
def create_agg_table(data_dict: dict):
    # Change data type of the data
    data_dict["products"]["ProductID"] = data_dict["products"]["ProductID"].astype(int)
    data_dict["products"]["Price"] = data_dict["products"]["Price"].astype(float)

    data_dict["orders"]["OrderID"] = data_dict["orders"]["OrderID"].astype(float)

    # Merge relevant table
    df = pd.merge(data_dict["products"], data_dict["orderdetails"], on="ProductID", how="inner")
    df = pd.merge(df, data_dict["orders"], on="OrderID", how="inner")

    # Extract Year and Month from OrderDate
    df["Year-Month"] = pd.to_datetime(df["OrderDate"])
    df["Year-Month"] = df["Year-Month"].dt.strftime("%Y-%m")

    # Calculate the sales_amount
    df["amount"] = df["Price"] * df["Quantity"]

    # 
    df = df.groupby(["Year-Month", "ProductID", "ProductName"]).sum("amount")

    # Compute the percentage_change
    df["percentage_change"] = df["amount"].pct_change() * 100

    return df